In [10]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm 

df = pd.read_csv(r'd:\naomy\LIA-FastAPI-MySQL\data\data.csv')
df.head()

,id,title,author,description,genre,classification,pages,owner_id
0,1,Matadouro cinco,Kurt Vonnegut,Um livro que zomba da estupidez humana com um ...,Science Fiction,5,288,1
1,2,Cama de gato,Kurt Vonnegut,Mais um livro incrível que zomba da condição h...,Science Fiction,5,280,1
2,3,Cem anos de solidão,Gabriel Garcia Marquez,Um realismo fantástico lindo e cativante,Fiction Novel,5,448,1
3,4,Cem anos de solidão,Gabriel Garcia Marquez,NaN,Fiction Novel,5,448,2
4,5,Sobre os ossos dos mortos,Olga Tockarzuck,Uma história sobre nossa relação com a naturez...,Fiction Novel,5,256,1


In [11]:
df_init = df.copy()
df_init.head()

,id,title,author,description,genre,classification,pages,owner_id
0,1,Matadouro cinco,Kurt Vonnegut,Um livro que zomba da estupidez humana com um ...,Science Fiction,5,288,1
1,2,Cama de gato,Kurt Vonnegut,Mais um livro incrível que zomba da condição h...,Science Fiction,5,280,1
2,3,Cem anos de solidão,Gabriel Garcia Marquez,Um realismo fantástico lindo e cativante,Fiction Novel,5,448,1
3,4,Cem anos de solidão,Gabriel Garcia Marquez,NaN,Fiction Novel,5,448,2
4,5,Sobre os ossos dos mortos,Olga Tockarzuck,Uma história sobre nossa relação com a naturez...,Fiction Novel,5,256,1


In [12]:
df.rename(columns={'id':'book_id'},inplace=True)
df.head()

,book_id,title,author,description,genre,classification,pages,owner_id
0,1,Matadouro cinco,Kurt Vonnegut,Um livro que zomba da estupidez humana com um ...,Science Fiction,5,288,1
1,2,Cama de gato,Kurt Vonnegut,Mais um livro incrível que zomba da condição h...,Science Fiction,5,280,1
2,3,Cem anos de solidão,Gabriel Garcia Marquez,Um realismo fantástico lindo e cativante,Fiction Novel,5,448,1
3,4,Cem anos de solidão,Gabriel Garcia Marquez,NaN,Fiction Novel,5,448,2
4,5,Sobre os ossos dos mortos,Olga Tockarzuck,Uma história sobre nossa relação com a naturez...,Fiction Novel,5,256,1


In [13]:
def normalize(data):
    '''
    Normalize input data to be between 0 and 1
    
    params:
        data: values you want to normalize
    
    returns:
        The input data normalized between 0 and 1
    '''
    min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    return [x/max_val for x in data]

In [14]:
df['pages_norm'] = normalize(df['pages'].values)
df['book_rating_norm'] = normalize(df['classification'].values)

In [15]:
def ohe(df, enc_col):
    '''
    This function will one hot encode the specified column and add it back
    onto the input dataframe
    
    params:
        df (DataFrame) : The dataframe you wish for the results to be appended to
        enc_col (String) : The column you want to OHE
    
    returns:
        The OHE columns added onto the input dataframe
    '''
    
    ohe_df = pd.get_dummies(df[enc_col])
    ohe_df.reset_index(drop = True, inplace = True)
    return pd.concat([df, ohe_df], axis = 1)

In [16]:
df = ohe(df = df, enc_col = 'genre')
df = ohe(df = df, enc_col = 'author')


In [17]:
cols = ['pages', 'genre', 'description', 'author', 'title', 'classification']
df.drop(columns = cols, inplace = True)
df.set_index('book_id', inplace = True)
df.head()

,owner_id,pages_norm,book_rating_norm,Dystopian,"Fantasy, Adventure",Fiction Novel,Graphic Novel,"Mystery, Suspense, Horror",Nonfiction,Science Fiction,...,Stephen King,Stephenie Meyer,Ted Chiang,Tess Gerritsen,Ursula K LeGuin,Ursula LeGuin,Victor Bonini,Victor LaValle,William Peter Blatty,William Shakespeare
book_id,,,,,,,,,,,,,,,,,,,,,
1,1,0.178549,1.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0.173590,1.0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0.277743,1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0.277743,1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0.158710,1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
class CBRecommend():
    def __init__(self, df):
        self.df = df
        
    def cosine_sim(self, v1,v2):
        '''
        This function will calculate the cosine similarity between two vectors
        '''
        return dot(v1,v2)/(norm(v1)*norm(v2))
    
    def recommend(self, book_id, n_rec, owner_id):
        """
        df (dataframe): The dataframe
        song_id (string): Representing the song name
        n_rec (int): amount of rec user wants
        """
        book_id = book_id - 1 #because it starts from zero, while the ids in the database start from 1
        inputVec = self.df.loc[book_id].values
        self.df['sim']= self.df.apply(lambda x: self.cosine_sim(inputVec,x.values), axis=1)
        
        df_rec = self.df.nlargest(columns='sim',n=n_rec)
        df_final = df_rec.loc[df['owner_id'] != owner_id]
        # returns top n user specified books that the user didnt listed already
        return df_final
#ver se troco o id pelo titulo 

In [19]:
t = df.copy()
cbr = CBRecommend(df = t)

df_recommendations = cbr.recommend(book_id = t.index[5], n_rec = 10, owner_id = 1)
df_recommendations

,owner_id,pages_norm,book_rating_norm,Dystopian,"Fantasy, Adventure",Fiction Novel,Graphic Novel,"Mystery, Suspense, Horror",Nonfiction,Science Fiction,...,Stephenie Meyer,Ted Chiang,Tess Gerritsen,Ursula K LeGuin,Ursula LeGuin,Victor Bonini,Victor LaValle,William Peter Blatty,William Shakespeare,sim
book_id,,,,,,,,,,,,,,,,,,,,,
4,2,0.277743,1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.757701
107,2,0.297582,1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.757680
84,2,0.130192,1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.756537
78,2,0.099194,1.0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.755995


In [20]:
df_recommendations.index.values

array([  4, 107,  84,  78], dtype=int64)

In [21]:
df_seila= df_init.loc[df_init['id'].isin(df_recommendations.index.values), ['title']]
df_seila.drop_duplicates(inplace=True)
#gets only the titles and the ids of the books recommended
df_seila.to_dict(orient='list')

{'title': ['Cem anos de solidão',
  'Crônicas de uma morte anunciada',
  'Tudo é rio',
  'A menina que roubava livros']}